### PyPDF

In [4]:
from langchain_community.document_loaders import PyPDFLoader

# Initialize the PDF loader
loader = PyPDFLoader(FILE_PATH)

# Load data into Document objects
docs = loader.load()

# Print the contents of the document
print(docs[10].page_content[:300])

Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEnc

xi
RDA: Recommended Daily Allowance
RPR:  Rapid Plasma Regain
S.PSA: Serum Prostate Specific Antigen
SLE: Systemic Lupus Eryth ematosus
TAPP: Trans Abdominal Preperitoneal Meshplasty
TEP: Totally Extra Peritoneal Meshplasty
TMJ: Tempor o M a ndibular Join t
TMT: Trabecular Meatel Technology
TPHA: Tr


In [8]:
docs[0].page_content

'i\nSTANDARD TREATMENT GUIDELINES\nA Manual for Medical Therapeutics\nFirst Edition, 2013\nGujarat Medical Services Corporation Limited\nHealth & Family Welfare Department\nGovernment of Gujarat'

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load Documents and split into chunks. Chunks are returned as Documents.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=200)
docs = loader.load_and_split(text_splitter=text_splitter)
print(docs[0].page_content)

In [8]:
import pymupdf4llm

md_text = pymupdf4llm.to_markdown("../data/raw/NewMedCollection/AcuteAltitudeSickness.pdf")

Processing ../data/raw/NewMedCollection/AcuteAltitudeSickness.pdf...
[                                        ] (0/1==[===                                     ] ( 1/1===[=======                                 ] ( 2/1==[==========                              ] ( 3/11===[==============                          ] ( 4/11===[==================                      ] ( 5/11==[=====================                   ] ( 6/1===[=========================               ] ( 7/1===[=============================           ] ( 8/1==[================================        ] ( 9/11===[====================================    ] (10/11==[========================================] (11/11]


In [9]:
import pathlib
pathlib.Path("output.md").write_bytes(md_text.encode())

60410

## Pdf to Markdown

The pdf is first converted to mark down using llama parse that can conserve table info as well as use ocr for better parsing.We implemented this method because We will use MarkdownHeaderTextSplitter first and then RecursiveCharacterText Splitter to better chunk daata.

In [10]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [16]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import os
import nest_asyncio

# Apply nest_asyncio to allow nested event loops (required in Jupyter notebooks or similar environments)
nest_asyncio.apply()

# Specify the file path
FILE_PATH = '../data/raw/NewMedCollection/AcuteAltitudeSickness.pdf'
print(f"Processing file: {FILE_PATH}")

# Configure the LlamaParse instance
parser = LlamaParse(
    result_type="markdown",  # Output format ("text", "markdown", "json", or "structured")
    num_workers=8,  # Number of workers for parallel processing
    verbose=True,  # Enable verbose logging
    language="en",  # Language of the document
    show_progress=True,  # Show progress bar
)

# Define a file extractor mapping file extensions to parsers
file_extractor = {".pdf": parser}

# Use SimpleDirectoryReader to parse the specified PDF file
try:
    documents = SimpleDirectoryReader(
        input_files=[FILE_PATH],  # List of files to process
        file_extractor=file_extractor,
    ).load_data()

    # Print the parsed documents
    for doc in documents:
        print(doc.text)  # Access the parsed text content
except Exception as e:
    print(f"Error loading or parsing the file: {e}")

Processing file: ../data/raw/NewMedCollection/AcuteAltitudeSickness.pdf
Started parsing the file under job_id ce6704ff-4e65-4f83-a424-740a5c824846
...# WILDERNESS MEDICAL SOCIETY PRACTICE GUIDELINES

# Wilderness Medical Society Practice Guidelines for the Prevention and Treatment of Acute Altitude Illness: 2014 Update

Andrew M. Luks, MD; Scott E. McIntosh, MD, MPH; Colin K. Grissom, MD; Paul S. Auerbach, MD, MS; George W. Rodway, PhD, APRN; Robert B. Schoene, MD; Ken Zafren, MD; Peter H. Hackett, MD

From the Division of Pulmonary and Critical Care Medicine, University of Washington, Seattle, WA (Drs Luks and Schoene); the Divisions of Emergency Medicine (Dr McIntosh); the Pulmonary and Critical Care Division, Intermountain Medical Center and the University of Utah, Salt Lake City, UT (Dr Grissom); the Department of Surgery, Division of Emergency Medicine, Stanford University School of Medicine, Stanford, CA (Drs Auerbach and Zafren); the Division of Health Sciences, University of Ne

In [21]:
md_file_path = "altitutesickness.md"
with open(md_file_path, "w", encoding="utf-8") as md_file:
            for doc in documents:
                # Write the text content of each document to the Markdown file
                md_file.write(doc.text + "\n\n")  # Add extra newlines for separation

In [23]:
## Script to convert parse al the pdf to makrdown format

from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import os

# Apply nest_asyncio to allow nested event loops (required in Jupyter notebooks or similar environments)
import nest_asyncio
nest_asyncio.apply()

# Define input and output directories
INPUT_DIR = "../data/raw/NewMedCollection"  # Directory containing PDF files
OUTPUT_DIR = "../data/parsed_markdown"  # Directory to save parsed Markdown files

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Configure the LlamaParse instance
parser = LlamaParse(
    result_type="markdown",  # Output format ("text", "markdown", "json", or "structured")
    num_workers=8,  # Number of workers for parallel processing
    verbose=True,  # Enable verbose logging
    language="en",  # Language of the document
    show_progress=True,  # Show progress bar
)

# Define a file extractor mapping file extensions to parsers
file_extractor = {".pdf": parser}

# Get a list of all PDF files in the input directory
pdf_files = [os.path.join(INPUT_DIR, f) for f in os.listdir(INPUT_DIR) if f.endswith(".pdf")]

# Process each PDF file
for pdf_file in pdf_files:
    try:
        print(f"Processing file: {pdf_file}")

        # Use SimpleDirectoryReader to parse the PDF file
        documents = SimpleDirectoryReader(
            input_files=[pdf_file],  # List of files to process
            file_extractor=file_extractor,
        ).load_data()

        # Extract the base name of the PDF file (without extension)
        base_name = os.path.splitext(os.path.basename(pdf_file))[0]

        # Define the output Markdown file path
        md_file_path = os.path.join(OUTPUT_DIR, f"{base_name}.md")

        # Write the parsed content to the Markdown file
        with open(md_file_path, "w", encoding="utf-8") as md_file:
            for doc in documents:
                # Write the text content of each document to the Markdown file
                md_file.write(doc.text + "\n\n")  # Add extra newlines for separation

        print(f"Saved parsed content to: {md_file_path}")

    except Exception as e:
        print(f"Error processing file {pdf_file}: {e}")

print("All files processed.")

Processing file: ../data/raw/NewMedCollection/gonorrhoeae.pdf
Started parsing the file under job_id 31f0eb82-f641-4dc5-bd80-6d0d78dd23f9
..Saved parsed content to: ../data/parsed_markdown/gonorrhoeae.md
Processing file: ../data/raw/NewMedCollection/Acne.pdf
Started parsing the file under job_id 53d7ca5d-5633-4911-bae1-d57aa85f6acd
.Saved parsed content to: ../data/parsed_markdown/Acne.md
Processing file: ../data/raw/NewMedCollection/nonalcoholic-fatty-liver-disease-nafld-assessment-and-management-pdf-1837461227461.pdf
Started parsing the file under job_id 82d0043f-1c39-41a1-8ab8-8d771c60960f
Saved parsed content to: ../data/parsed_markdown/nonalcoholic-fatty-liver-disease-nafld-assessment-and-management-pdf-1837461227461.md
Processing file: ../data/raw/NewMedCollection/metlay-et-al-2019-diagnosis-and-treatment-of-adults-with-community-acquired-pneumonia-an-official-clinical-practice.pdf
Started parsing the file under job_id ed1ff7b9-67c0-4910-9564-2d9cd656b85d
.Saved parsed content to:

## MarkdownHeaderTextSplitter and Combining with Other Text Splitters

In [29]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

file_path = '../data/parsed_markdown/AcuteAltitudeSickness.md'
with open(file_path, "r", encoding="utf-8") as md_file:
    
    markdown_document = md_file.read()



# Define a markdown document as a string
print(markdown_document[:100])

# WILDERNESS MEDICAL SOCIETY PRACTICE GUIDELINES

# Wilderness Medical Society Practice Guidelines f


In [36]:
headers_to_split_on = [  # Define header levels and their names for document splitting
    (
        "#",
        "Header 1",
    ),  # Header level 1 is marked with '#' and named 'Header 1'
    (
        "##",
        "Header 2",
    ),  # Header level 2 is marked with '##' and named 'Header 2'
    (
        "###",
        "Header 3",
    ),  # Header level 3 is marked with '###' and named 'Header 3'
]

# Create a MarkdownHeaderTextSplitter object to split text based on markdown headers
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
# Split markdown_document by headers and store in md_header_splits
md_header_splits = markdown_splitter.split_text(markdown_document)


# Filter splits with fewer than 10 words and reformat metadata
filtered_splits = []
for split in md_header_splits:
    # Count the number of words in the split
    word_count = len(split.page_content.split())

    # If the split has 10 or more words, keep it
    if word_count >= 10:
        # Reformat metadata (if needed)
        filtered_splits.append(split)

# Print the filtered splits
for split in filtered_splits:
    print(f"Content: {split.page_content}")
    print(f"Metadata: {split.metadata}")
    print("=====================")

Content: Andrew M. Luks, MD; Scott E. McIntosh, MD, MPH; Colin K. Grissom, MD; Paul S. Auerbach, MD, MS; George W. Rodway, PhD, APRN; Robert B. Schoene, MD; Ken Zafren, MD; Peter H. Hackett, MD  
From the Division of Pulmonary and Critical Care Medicine, University of Washington, Seattle, WA (Drs Luks and Schoene); the Divisions of Emergency Medicine (Dr McIntosh); the Pulmonary and Critical Care Division, Intermountain Medical Center and the University of Utah, Salt Lake City, UT (Dr Grissom); the Department of Surgery, Division of Emergency Medicine, Stanford University School of Medicine, Stanford, CA (Drs Auerbach and Zafren); the Division of Health Sciences, University of Nevada, Reno, NV (Dr Rodway); the East Bay Regional Pulmonary and Critical Care Medicine Associates, Berkeley, CA (Dr Schoene); the Himalayan Rescue Association, Kathmandu, Nepal (Dr Zafren); and the Division of Emergency Medicine, Altitude Research Center, University of Colorado School of Medicine, Aurora and th

In [37]:
## Combining with other text splitters


from langchain_text_splitters import RecursiveCharacterTextSplitter


chunk_size = 200  # Specify the size of each split chunk
chunk_overlap = 20  # Specify the number of overlapping characters between chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split the document into chunks by characters
splits = text_splitter.split_documents(filtered_splits)
# Print the split results
for header in splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

Andrew M. Luks, MD; Scott E. McIntosh, MD, MPH; Colin K. Grissom, MD; Paul S. Auerbach, MD, MS; George W. Rodway, PhD, APRN; Robert B. Schoene, MD; Ken Zafren, MD; Peter H. Hackett, MD
{'Header 1': 'Wilderness Medical Society Practice Guidelines for the Prevention and Treatment of Acute Altitude Illness: 2014 Update'}
From the Division of Pulmonary and Critical Care Medicine, University of Washington, Seattle, WA (Drs Luks and Schoene); the Divisions of Emergency Medicine (Dr McIntosh); the Pulmonary and Critical
{'Header 1': 'Wilderness Medical Society Practice Guidelines for the Prevention and Treatment of Acute Altitude Illness: 2014 Update'}
and Critical Care Division, Intermountain Medical Center and the University of Utah, Salt Lake City, UT (Dr Grissom); the Department of Surgery, Division of Emergency Medicine, Stanford University
{'Header 1': 'Wilderness Medical Society Practice Guidelines for the Prevention and Treatment of Acute Altitude Illness: 2014 Update'}
Stanford Unive

In [8]:
from langchain_text_splitters import (
    MarkdownHeaderTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import os

# Configuration
CHUNK_SIZE = 200
CHUNK_OVERLAP = 20
EMBEDDING_MODEL = "text-embedding-3-small"  # or "text-embedding-3-large"
PERSIST_DIRECTORY = "./chroma_db"

# Initialize components
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
)
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

def process_markdown_files(md_files):
    """Process multiple markdown files into ChromaDB index"""
    all_splits = []
    
    for md_file in md_files:
        with open(md_file, "r", encoding="utf-8") as f:
            markdown_document = f.read()
        
        # Step 1: Split by headers
        header_splits = markdown_splitter.split_text(markdown_document)
        
        # Step 2: Filter small chunks
        filtered_splits = [
            split for split in header_splits 
            if len(split.page_content.split()) >= 10
        ]
        
        # Step 3: Further split by character
        final_splits = text_splitter.split_documents(filtered_splits)
        
        # Add source file metadata
        for split in final_splits:
            split.metadata["source"] = os.path.basename(md_file)
        
        all_splits.extend(final_splits)
    
    return all_splits

def create_chroma_index(splits, persist_directory=PERSIST_DIRECTORY):
    """Create ChromaDB index from processed splits"""
    return Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_directory=persist_directory
    )

# def main(md_files):
#     # Process all markdown files
#     processed_splits = process_markdown_files(md_files)
    
#     # Create and persist ChromaDB index
#     vectorstore = create_chroma_index(processed_splits)
    
#     # Save index to disk
#     vectorstore.persist()
    
#     return vectorstore

# # Example Usage
# if __name__ == "__main__":
#     # Get list of markdown files
#     md_files = ['../temp/AcuteAltitudeSickness.md']
    
#     # Create and persist index
#     vectorstore = main(md_files)
    
#     print(f"Index created with {len(vectorstore.get()['documents'])} chunks")
#     print(f"Persisted to {PERSIST_DIRECTORY}")

In [23]:
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
retriever = vectorstore.similarity_search_with_relevance_scores(search_kwargs={"k": 3}) 

def retrieve_with_scores(query, k=3):
    return vectorstore.similarity_search_with_score(query, k=k)

TypeError: VectorStore.similarity_search_with_relevance_scores() missing 1 required positional argument: 'query'

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI

prompt_template = ChatPromptTemplate.from_template("""
You are an expert at answering multiple-choice questions (MCQs). Use the following context to answer the question.

Context:
{context}

Question:
{question}

Options:
{options}

Provide the correct answer as a single letter (A-D) and a detailed reasoning for your choice.
Correct Answer: 
Reasoning:
""")

In [11]:
# Step 3: Define the model and output parser
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
output_parser = StrOutputParser()

# Step 4: Create the LCEL chain with retriever
chain = (
    {
        "context": RunnableLambda(lambda x: retriever.get_relevant_documents(x["question"])),
        "question": RunnablePassthrough(),
        "options": RunnableLambda(lambda x: "\n".join([f"{i+1}. {opt}" for i, opt in enumerate(x["options"])])),
    }
    | prompt_template
    | model
    | output_parser
)

In [18]:
# Sample questions to test retrieval
test_questions = [
    "A 6-year-old child with a history of recurrent wheezing, triggered by upper respiratory infections, presents to your clinic. The child's parent reports that the wheezing episodes have been occurring approximately every 2-3 months, significantly disrupting the child's sleep and daily activities. The child's current treatment consists of a short-acting beta2-agonist (SABA) for symptom relief as needed. Based on the guidelines provided, which of the following is the MOST appropriate next step in managing this child's asthma, considering both the potential benefits and harms of the treatment?",
    "How is hypertension managed?",
    "What are the symptoms of pneumonia?",
]

# Test retrieval for each question
for question in test_questions:
    print(f"\nQuestion: {question}")
    retrieved_docs_with_scores = retrieve_with_scores(question)
    
    for i, (doc, score) in enumerate(retrieved_docs_with_scores):
        print(f"\nRetrieved Document {i+1}:")
        print(f"Content: {doc.page_content}")
        print(f"Metadata: {doc.metadata}")
        print(f"Similarity Score: {score:.4f}")
        print("=" * 50)


Question: A 6-year-old child with a history of recurrent wheezing, triggered by upper respiratory infections, presents to your clinic. The child's parent reports that the wheezing episodes have been occurring approximately every 2-3 months, significantly disrupting the child's sleep and daily activities. The child's current treatment consists of a short-acting beta2-agonist (SABA) for symptom relief as needed. Based on the guidelines provided, which of the following is the MOST appropriate next step in managing this child's asthma, considering both the potential benefits and harms of the treatment?

Retrieved Document 1:
Content: In a single randomized, placebo-controlled study, the long-acting inhaled β-agonist salmeterol decreased the incidence of HAPE by 50% in susceptible individuals.46 Very high doses (125 μg twice daily)
Metadata: {'Header 1': 'Salmeterol', 'source': 'AcuteAltitudeSickness.md'}
Similarity Score: 0.9570

Retrieved Document 2:
Content: Although there are reports o

In [24]:
import json

# Load questions from JSON
with open("question_asthma.json", "r") as f:
    questions_data = json.load(f)

# Process each question
results = []
for question_data in questions_data[0:1]:  # Process only the first 2 questions for testing
    try:
        # Retrieve context
        retrieved_docs = retriever.invoke(question_data["question"])
        context = "\n".join([doc.page_content for doc in retrieved_docs])
        
        # Print retrieved context
        print(f"\nRetrieved Context for Question: {question_data['question']}")
        print(context)
        print("=" * 50)
        
        # Invoke the chain
        response = chain.invoke(question_data)
        
        # Parse the response
        model_answer = response.split("Correct Answer:")[1].split()[0].strip()
        reasoning = response.split("Reasoning:")[1].strip()
        
        # Store results
        results.append({
            "question": question_data["question"],
            "context": context,  # Include retrieved context in results
            "model_answer": model_answer,
            "reasoning": reasoning,
        })
    except Exception as e:
        print(f"Error processing question: {question_data['question']}")
        print(f"Error: {str(e)}")

# Save results
with open("results1.json", "w") as f:
    json.dump(results, f, indent=2)


Retrieved Context for Question: A 45-year-old patient, Ms. Roberts, presents with a history of moderate persistent asthma. She has been on medium-dose inhaled corticosteroids (ICS) and a long-acting beta2-agonist (LABA) for the past year. Her symptoms are well controlled most of the time, but she experiences worsening symptoms during seasonal changes. She is interested in learning about additional measures to reduce seasonal exacerbations. Ms. Roberts has no history of smoking, and spirometry shows moderate obstruction with significant reversibility. Allergy testing reveals sensitization to dust mites and pollen. What would be the most appropriate next step in her management to address her seasonal exacerbations?
In a single randomized, placebo-controlled study, the long-acting inhaled β-agonist salmeterol decreased the incidence of HAPE by 50% in susceptible individuals.46 Very high doses (125 μg twice daily)
Before initiating treatment, care should be taken to rule out other causes 